In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

# model/ フォルダからロード
tokenizer = AutoTokenizer.from_pretrained("model")
model = AutoModelForSequenceClassification.from_pretrained("model")

In [ ]:
# 推論用テキストをトークナイズ
import torch

text = "This movie had great direction and acting."

# トークナイズ
inputs = tokenizer(text, return_tensors="pt", padding=True, truncation=True)

# 推論モードへ（勾配不要）
model.eval()
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_id = logits.argmax(dim=-1).item()

In [ ]:
# モデル保存時にconfigに含めていれば読み出せる
label_name = model.config.id2label[predicted_id]
print("Predicted label:", label_name)

In [ ]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
import torch

# モデルとトークナイザーの再ロード
tokenizer = AutoTokenizer.from_pretrained("model")
model = AutoModelForSequenceClassification.from_pretrained("model")
model.eval()

# 複数テキスト
texts = [
    "The movie was fantastic and well-acted.",
    "This lipstick is a great shade of red.",
    "Electric cars are the future of transportation."
]

# トークナイズ（バッチ対応）
inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)

# 推論
with torch.no_grad():
    outputs = model(**inputs)
    logits = outputs.logits
    predicted_ids = torch.argmax(logits, dim=1).tolist()

# ラベル名に変換（configにid2labelがある場合）
labels = [model.config.id2label[i] for i in predicted_ids]
# 信頼度
probs = torch.nn.functional.softmax(logits, dim=1)
confidences = probs.max(dim=1).values.tolist()
# 結果表示
for text, label, conf in zip(texts, labels, confidences):
    print(f"Text: {text}\n→ Predicted label: {label} (confidence: {conf:.2f})\n")